In [1]:
import os
import sys
import time
import warnings
os.system('clear')
warnings.filterwarnings("ignore")

import torch
import torch.optim as optim
# import torch.utils.data as data

import importlib

In [2]:
sys.path.append('../Models/')
sys.path.append('/remote/tychodata/ftairli/work/Projecs/Common/')



In [3]:
# Load Dataset, assuming its already been calculated
if 'ProcDS' not in globals():
    ProcDS = torch.load('/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Data/XmaxE_Conv3d_Dataset.pt')
    ProcDS.AssignIndices()

ProcDS.BatchSize = 64
ProcDS.RandomIter = True

In [8]:
T = torch.zeros([10000,1000000])

In [4]:
# Define Some Paths
SavePath     = '/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Models/'
plotSavePath = '/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Results/TrainingPlots/'
LogPath      = '/remote/tychodata/ftairli/work/Projects/TrainingLogs/'

if plotSavePath != None:  # Purge the directory
        os.system(f'rm -r {plotSavePath}')
        os.system(f'mkdir {plotSavePath}')



In [19]:
# Load the model and axuillary functions
ModelPath = '/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Models/'
sys.path.append(ModelPath)

import TrainingModule
importlib.reload(TrainingModule)
Tracker = TrainingModule.Tracker
Train   = TrainingModule.Train



import Model_XmaxE_Conv as ModelsFile
importlib.reload(ModelsFile)

Loss_function = ModelsFile.Loss
validate      = ModelsFile.validate
metric        = ModelsFile.metric

Model         = ModelsFile.Model_XmaxE_Conv_3d_Distances_JustXmax


# Setup the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Model Parameters
Model_Parameters = {
    'in_main_channels': (3,),
    'in_node_channels': 5   ,
    'in_edge_channels': 2   ,
    'in_aux_channels' : 1   ,
    'N_kernels'       : 2   ,
    'N_Graph_Heads'   : 2   ,
    'N_dense_nodes'   : 16  ,
    'N_LSTM_nodes'    : 64  ,
    'N_LSTM_layers'   : 5   ,
    'kernel_size'     : 10  ,
    'conv2d_init_type': 'normal',
    'model_Dropout'   : 0.2
    }


Training_Parameters = {
        'LR': 0.0001,
        'epochs': 5,
        'BatchSize': ProcDS.BatchSize, # In notebook set during loading
        'accumulation_steps': 1,
        'epoch_done': 0,
        'batchBreak': 1e99,
        'ValLossIncreasePatience': 15,
        'Optimizer': 'Adam'
    }


model = Model(**Model_Parameters).to(device)

if Training_Parameters['Optimizer'] == 'Adam': optimizer = optim.Adam(model.parameters(), lr=Training_Parameters['LR'])
if Training_Parameters['Optimizer'] == 'SGD' : optimizer = optim.SGD (model.parameters(), lr=Training_Parameters['LR'], momentum=0.9)
# Define scheduler
gamma = 0.001**(1/30) if Training_Parameters['epochs']>30 else 0.001**(1/Training_Parameters['epochs']) # Reduce the LR by factor of 1000 over 30 epochs or less
scheduler = torch.optim.lr_scheduler.ExponentialLR    (optimizer, gamma = gamma, last_epoch=-1, verbose=False)



In [20]:
model,tracker = Train(model,ProcDS,optimizer,scheduler,Loss_function,validate,metric ,Tracker,device = device,\
                      plotOnEpochCompletionPath=None,Training_Parameters=Training_Parameters,Model_Parameters=Model_Parameters,LogPath=LogPath)
        

Epoch 1/5
Early Exit for Testingss: 1.1340

Calculating Val Metrics
torch.Size([512, 2])
torch.Size([512, 2])


TypeError: 'NoneType' object is not subscriptable

In [16]:
print(ProcDS.Unnormalise_Truth)

<function Unnormalise_Xmax_Energy at 0x7f9b39275f70>


In [7]:
def New_Lout(Lin,Kernel,Stride,Padding,dilation):
    '''
    Takes Lin,Kernel,Stride,Padding,dilation
    Returns the output size of the Conv1d layer
    '''
    return int((Lin + 2*Padding - dilation*(Kernel-1) - 1)/Stride + 1)


KernelSize = 10
Stride     = 2 
Padding    = 1
dilation   = 1

Lout = 500
print(Lout)
Lout = New_Lout(Lout,KernelSize,Stride,Padding,dilation)
print(Lout)
Lout = New_Lout(Lout,KernelSize,Stride,Padding,dilation)
print(Lout)
Lout = New_Lout(Lout,KernelSize,Stride,Padding,dilation)
print(Lout)


500
247
120
57


In [8]:
ProcDS.BatchSize = 1
from torch_geometric.nn import max_pool_x

out = []

for _,BatchGraph,BatchAux,BatchTruth,_ in ProcDS:
    device = model.Chi01.weight.device
        
    # UnloadGraph
    AllNodes = []
    AllEdges = []
    AllEdgeV = []
    # EvSize   = [] # EvSize is no longer required
    Batching = []
    TotalNNodes = 0
    

    for BatchI,(Nodes,Edges,EdgeV) in enumerate(BatchGraph):
        AllNodes.append(Nodes)
        AllEdges.append(Edges+TotalNNodes)
        AllEdgeV.append(EdgeV)
        
        # EvSize.append(len(torch.unique(Nodes[:,2]))) # number of unique time steps in the event
        Batching.append(torch.ones(Nodes.shape[0])*BatchI)
        TotalNNodes += Nodes.shape[0]


    AllNodes = torch.cat(AllNodes,dim=0).to(device)
    AllEdges = torch.cat(AllEdges,dim=0).to(device).T
    AllEdgeV = torch.cat(AllEdgeV,dim=0).to(device)

    # Node Info
    # EvSize   = torch.tensor(EvSize)     .to(device).requires_grad_(False)
    Batching = torch.cat(Batching,dim=0).to(device).requires_grad_(False).to(torch.long)
    Timing   = AllNodes[:,2]                       .requires_grad_(False).to(torch.long)

    
    # Graph Convolution
    out.append(model.Graph1(AllNodes,AllEdges,edge_attr=AllEdgeV))
    
    # Do the Summary by masking edges with NBL >1
    MaskedEdges = AllEdges.T[(AllEdgeV[:,0] > 1)].T
    MaskedEdgeV = AllEdgeV[(AllEdgeV[:,0] > 1)]
    out.append(model.GraphS(out[-1],MaskedEdges,edge_attr=MaskedEdgeV))
    Max_out,_ = max_pool_x(cluster = Timing, x = out[-1], batch = Batching, size = 500)
    
    # Reshape for Conv1d
    out.append(Max_out.view(-1,out[-1].shape[1],500))
    print(out[-1].shape)
    
    out.append(model.ConvActivation(model.Conv1(out[-1])))
    print(out[-1].shape)
    out.append(model.ConvActivation(model.Conv2(out[-1])))
    print(out[-1].shape)
    out.append(model.ConvActivation(model.Conv3(out[-1])))
    print(out[-1].shape)
    
    Max_out = model.MaxPool(out[-1])
    print(Max_out.shape)
    # Dense Layers
    out.append(Max_out.view(Max_out[-1].shape[0],-1))
    print(out[-1].shape)
    
    # Output Layers
    Chi0 = model.DenseActivation(model.Chi01(out[-1] ))
    Chi0 = model.DenseActivation(model.Chi02(Chi0))
    Chi0 = model.AngleActivation(model.Chi03(Chi0))
    
    Rp = model.DenseActivation(model.Rp1(out[-1]))
    Rp = model.DenseActivation(model.Rp2(Rp ))
    Rp =                      model.Rp3(Rp )

    T0 = model.DenseActivation(model.T01(out[-1]))
    T0 = model.DenseActivation(model.T02(T0 ))
    T0 =                      model.T03(T0 )



    break

RuntimeError: expected mat1 and mat2 to have the same dtype, but got: long int != float

In [ ]:
torch.cuda.empty_cache()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
